In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from scipy.sparse.linalg import svds

# Function to create a collaborative filtering model using SVD
def investment_recommendation(data, user_id, num_recommendations=5):
    # Create the investment matrix
    investment_matrix = data.pivot(index='user_id', columns='project_id', values='investment_amount').fillna(0)
    investment_matrix_np = investment_matrix.to_numpy()

    # Perform SVD
    k = min(investment_matrix_np.shape) - 1  # Set number of latent factors
    U, sigma, Vt = svds(investment_matrix_np, k=k)
    sigma = np.diag(sigma)

    # Predict missing investment amounts
    predicted_investments = np.dot(np.dot(U, sigma), Vt)
    predicted_df = pd.DataFrame(predicted_investments, columns=investment_matrix.columns, index=investment_matrix.index)

    # Recommend projects for the specified user
    if user_id not in predicted_df.index:
        print(f"User ID {user_id} not found.")
        return {}

    user_predictions = predicted_df.loc[user_id].sort_values(ascending=False)
    recommendations = {project: round(user_predictions[project], 2) for project in user_predictions.head(num_recommendations).index}
    return recommendations  

# Generate random data for demonstration
np.random.seed(42)
user_ids = np.arange(1, 11)
project_ids = np.arange(1, 6)
data = []

for user in user_ids:
    for project in project_ids:
        investment = np.random.choice([0, np.random.randint(1, 100)], p=[0.6, 0.4])  # 60% chance of 0
        data.append([user, project, investment])

# Create a DataFrame
df = pd.DataFrame(data, columns=['user_id', 'project_id', 'investment_amount'])

# Specify user ID for recommendations
user_id = 1
recommendations = investment_recommendation(df, user_id)

# Display recommendations
if recommendations:
    print(f"Top recommended projects and investment amounts for user {user_id}:")
    for project, investment in recommendations.items():
        print(f"Project ID: {project}, Predicted Investment Amount: {investment}")
